In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
houseData = pd.read_csv("C:\\Users\\cbt04\\Desktop\\train (1).csv")

features = houseData[["GrLivArea", "YearBuilt"]]
target = houseData["SalePrice"]
features = StandardScaler().fit_transform(features, target)
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.3)

Blending

Regression analysis using blending

In [6]:
model1, model2, model3 = LinearRegression(), SVR(kernel="linear", C=0.8), DecisionTreeRegressor(criterion="squared_error", max_depth=5)

models = [model1, model2, model3]

for model in models:
    model.fit(x_train, y_train)

pred1 = model1.predict(x_test)
pred2 = model2.predict(x_test)
pred3 = model3.predict(x_test)

x_merge = np.concatenate([pred1.reshape(-1, 1), pred2.reshape(-1, 1), pred3.reshape(-1, 1)], axis=1)


model4 = VotingRegressor([("Linear Regression", model1), ("SVR", model2), ("DT", model3)])
pred4 = model4.fit(x_merge, y_test).predict(x_merge)


for model in models:
    print(f"{model.__class__.__name__}: {model.score(x_test, y_test)}")

print(f"{model4.__class__.__name__}: {model4.score(x_merge, y_test)}")

LinearRegression: 0.6080748218426015
SVR: -0.025083232136626554
DecisionTreeRegressor: 0.6229412717961806
VotingRegressor: 0.7734193739965904


Classification analysis using blending

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import load_iris

In [8]:
features= load_iris(as_frame=True).data
target = load_iris(as_frame=True).target

data = pd.concat([features, target], axis=1)

data = data[data["target"] != 2]

features = data[["sepal length (cm)", "petal length (cm)"]]
target = data["target"]

features = StandardScaler().fit_transform(features, target)
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.3)

In [10]:
model1, model2, model3 = LogisticRegression(), SVC(kernel="linear", C=0.8, probability=True), DecisionTreeClassifier(criterion="gini", max_depth=5)

models = [model1, model2, model3]

for model in models:
    model.fit(x_train, y_train)

pred1 = model1.predict_proba(x_test)[:, 0]
pred2 = model2.predict_proba(x_test)[:, 0]
pred3 = model3.predict_proba(x_test)[:, 0]

x_merge = np.concatenate([pred1.reshape(-1, 1), pred2.reshape(-1, 1), pred3.reshape(-1, 1)], axis=1)

model4 = VotingClassifier([("Linear Regression", model1), ("SVR", model2), ("DT", model3)], voting="hard")
pred4 = model4.fit(x_merge, y_test).predict(x_merge)


for model in models:
    print(f"{model.__class__.__name__}: {model.score(x_test, y_test)}")

print(f"{model4.__class__.__name__}: {model4.score(x_merge, y_test)}")

LogisticRegression: 1.0
SVC: 1.0
DecisionTreeClassifier: 1.0
VotingClassifier: 1.0



Bagging

In [11]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

dtc = DecisionTreeClassifier(criterion="entropy")
model = BaggingClassifier(dtc, 20).fit(x_train, y_train)
knn = KNeighborsClassifier(n_neighbors= 7).fit(x_train, y_train)

print(f"{model.__class__.__name__}: {model.score(x_test, y_test)}")
print(f"{knn.__class__.__name__}: {knn.score(x_test, y_test)}")

BaggingClassifier: 1.0
KNeighborsClassifier: 1.0


In [12]:
confusion_matrix(knn.predict(x_test), y_test)

array([[13,  0],
       [ 0, 17]], dtype=int64)


Stacking

In [15]:
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score

svc = SVC(C=0.6, kernel="poly")
tree = DecisionTreeClassifier(criterion="log_loss")
knn = KNeighborsClassifier(n_neighbors=7)


svc.fit(x_train, y_train)
tree.fit(x_train, y_train)
knn.fit(x_train, y_train)

svc_pred = svc.predict(x_test)
tree_pred = tree.predict(x_test)
knn_pred = knn.predict(x_test)

meta_train = np.column_stack([svc_pred, tree_pred, knn_pred])

meta = LogisticRegression().fit(meta_train, y_test)
meta_new = meta.predict(meta_train)


print(f"Scratch stacked accuracy: {accuracy_score(y_test, meta_new)}")

Scratch stacked accuracy: 1.0


In [16]:
svc = SVC(C=0.6, kernel="poly")
tree = DecisionTreeClassifier(criterion="log_loss")
knn = KNeighborsClassifier(n_neighbors=7)

mod = StackingClassifier([("svc", svc), ("tree", tree), ("knn", knn)], final_estimator=LogisticRegression())

mod.fit(x_train, y_train)

score = mod.score(x_test, y_test)

print(f"Sklearn stacked accuracy: {score}")

Sklearn stacked accuracy: 1.0
